# 使用BaseMap绘制地图

#### —— nmc_met_graphics程序库使用说明

国家气象中心天气预报技术研发室    
Sep, 2020    
Kan Dai 

nmc_met_graphics提供BaseMap类, 用于方便地绘制中国地图.

* 对Cartopy地图绘制进行封装
* 提供中国国界, 省界, 河流和县界等的绘制
* 提供对国家或省份区域外的白化
* 提供天地, Arcgis等背景图

## Prepare

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# load libraries
import matplotlib.pyplot as plt
import cartopy